In [1]:
import cv2
import numpy as np

# Load the pre-trained MobileNet SSD model and the class labels
net = cv2.dnn.readNetFromCaffe("C:\\Users\\vashi\\OneDrive\\Desktop\\women\\deploy.prototxt", "C:\\Users\\vashi\\OneDrive\\Desktop\\women\\mobilenet_iter_73000 (2).caffemodel")
classes = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus",
           "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike",
           "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

# Start capturing video
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    h, w = frame.shape[:2]

    # Preprocess the frame for object detection
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    # Loop over the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])
            if classes[idx] == "person":
                # Draw a bounding box around the detected person
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                label = f"Person: {confidence:.2f}"
                cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # Break the loop on 'q' key press
    if key == ord("q"):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


#GENDER CLASSIFICATION MODEL 

In [3]:
import cv2

def faceBox(faceNet, frame):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (227, 227), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bboxs = []
    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detection[0, 0, i, 3] * frameWidth)
            y1 = int(detection[0, 0, i, 4] * frameHeight)
            x2 = int(detection[0, 0, i, 5] * frameWidth)
            y2 = int(detection[0, 0, i, 6] * frameHeight)
            bboxs.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    return frame, bboxs

facePhoto = "opencv_face_detector.pbtxt"
face_Model = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

faceNet = cv2.dnn.readNet(face_Model, facePhoto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = [ 'Male','Female']

video = cv2.VideoCapture(0)

padding=20

while True:
    ret, frame = video.read()
    if not ret:
        break

    frame, bboxs = faceBox(faceNet, frame)
    for bbox in bboxs:
       # face = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        face=frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding,frame.shape[1]-1)]
        if face.size == 0:
            continue
        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPrediction = genderNet.forward()
        gender = genderList[genderPrediction[0].argmax()]

        ageNet.setInput(blob)
        agePred = ageNet.forward()
        age = ageList[agePred[0].argmax()]

        label = "{}, {}".format(gender, age)
        cv2.rectangle(frame,(bbox[0],bbox[1]-30),(bbox[2],bbox[1]),(0,255,0),-1)
        cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2,cv2.LINE_AA)
    
    cv2.imshow("Age_gender", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


##ANOMALY DETECTION 

In [3]:
from datetime import datetime
import cv2
import numpy as np

def classify_gender(face):
    # Implement gender classification logic using a pre-trained model
    # For demonstration, we will use a dummy implementation
    # Replace this with actual model inference code
    return "Female"  # or "Male", depending on your actual implementation

def is_lone_woman_detected(gender_counts):
    current_hour = datetime.now().hour
    if current_hour >= 20 or current_hour <= 6:  # Consider night hours
        if gender_counts["Female"] == 1 and gender_counts["Male"] == 0:
            return True
    return False

# Load models and initialize
video = cv2.VideoCapture(0)  # Use 'video' as the capture variable

# Ensure these paths are correct and the files exist
face_prototxt = "C:\\Users\\vashi\\OneDrive\\Desktop\\women\\deploy.prototxt"
face_model = "C:\\Users\\vashi\\OneDrive\\Desktop\\women\\mobilenet_iter_73000 (2).caffemodel"

try:
    net = cv2.dnn.readNetFromCaffe(face_prototxt, face_model)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

classes = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", 
           "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike",
           "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

while True:
    ret, frame = video.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    
    try:
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()
    except Exception as e:
        print(f"Error during detection: {e}")
        continue

    gender_counts = {"Male": 0, "Female": 0}

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(detections[0, 0, i, 1])
            if classes[idx] == "person":
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                face = frame[startY:endY, startX:endX]

                gender = classify_gender(face)
                if gender in gender_counts:
                    gender_counts[gender] += 1
                else:
                    print(f"Unexpected gender value: {gender}")

                label = f"{gender} ({confidence:.2f})"
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2,cv2.LINE_AA)

    if is_lone_woman_detected(gender_counts):
        cv2.putText(frame, "ALERT: Lone Woman Detected!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video.release()
cv2.destroyAllWindows()


full complete model 

In [2]:
from datetime import datetime
import cv2
import numpy as np

def classify_gender(face, genderNet, genderList):
    if face.size == 0:
        return "Unknown"
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPrediction = genderNet.forward()
    return genderList[genderPrediction[0].argmax()]

def classify_age(face, ageNet, ageList):
    if face.size == 0:
        return "Unknown"
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    ageNet.setInput(blob)
    agePred = ageNet.forward()
    return ageList[agePred[0].argmax()]

def is_lone_woman_detected(gender_counts):
    current_hour = datetime.now().hour
    if current_hour >= 20 or current_hour <= 6:  # Consider night hours
        if gender_counts["Female"] == 1 and gender_counts["Male"] == 0:
            return True
    return False

def faceBox(faceNet, frame):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (227, 227), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bboxs = []
    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detection[0, 0, i, 3] * frameWidth)
            y1 = int(detection[0, 0, i, 4] * frameHeight)
            x2 = int(detection[0, 0, i, 5] * frameWidth)
            y2 = int(detection[0, 0, i, 6] * frameHeight)
            bboxs.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    return frame, bboxs

# Load models and initialize
facePhoto = "opencv_face_detector.pbtxt"
face_Model = "opencv_face_detector_uint8.pb"
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

faceNet = cv2.dnn.readNet(face_Model, facePhoto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

video = cv2.VideoCapture(0)

padding = 20

while True:
    ret, frame = video.read()
    if not ret:
        break

    frame, bboxs = faceBox(faceNet, frame)
    gender_counts = {"Male": 0, "Female": 0}

    for bbox in bboxs:
        face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
                     max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
        
        gender = classify_gender(face, genderNet, genderList)
        age = classify_age(face, ageNet, ageList)

        if gender != "Unknown":
            gender_counts[gender] += 1

        label = "{}, {}".format(gender, age)
        cv2.rectangle(frame, (bbox[0], bbox[1] - 30), (bbox[2], bbox[1]), (0, 255, 0), -1)
        cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    if is_lone_woman_detected(gender_counts):
        cv2.putText(frame, "ALERT: Lone Woman Detected!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Age_gender", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
from datetime import datetime

# Load models for face detection, age, and gender classification
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
gestureModel = "gesture_recognition_model.h5"  # Placeholder for gesture recognition model

faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
gestureNet = None  # Load gesture recognition model if available

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Function to detect faces
def faceBox(faceNet, frame):
    frameHeight, frameWidth = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bboxs = []
    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detection[0, 0, i, 3] * frameWidth)
            y1 = int(detection[0, 0, i, 4] * frameHeight)
            x2 = int(detection[0, 0, i, 5] * frameWidth)
            y2 = int(detection[0, 0, i, 6] * frameHeight)
            bboxs.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    return frame, bboxs

# Function to classify gender
def classify_gender(face, genderNet, genderList):
    if face.size == 0:
        return "Unknown"
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPred = genderNet.forward()
    return genderList[genderPred[0].argmax()]

# Function to detect gestures (Placeholder)
def detect_gesture(frame, gestureNet):
    # Placeholder function for gesture detection
    # Implement your gesture detection model here
    # Example: Return 'Suspicious' if a threatening gesture is detected
    return "Normal"  # Default to 'Normal' if no gesture model is available

# Function to check for lone woman
def is_lone_woman_detected(gender_counts):
    current_hour = datetime.now().hour
    if (current_hour >= 20 or current_hour <= 6) and gender_counts['Female'] == 1 and gender_counts['Male'] == 0:
        return True
    return False

# Function to check for anomalies (Placeholder)
def detect_anomalies(frame, gender_counts):
    # Placeholder for advanced anomaly detection
    # Implement your anomaly detection algorithm here
    return False  # Return True if anomaly is detected

# Real-time video capture
video = cv2.VideoCapture(0)

padding = 20

while True:
    ret, frame = video.read()
    if not ret:
        break

    frame, bboxs = faceBox(faceNet, frame)
    gender_counts = {"Male": 0, "Female": 0}

    for bbox in bboxs:
        face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
                     max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
        
        gender = classify_gender(face, genderNet, genderList)
        if gender in gender_counts:
            gender_counts[gender] += 1

        label = f"{gender}"
        cv2.rectangle(frame, (bbox[0], bbox[1] - 30), (bbox[2], bbox[1]), (0, 255, 0), -1)
        cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    if is_lone_woman_detected(gender_counts):
        cv2.putText(frame, "ALERT: Lone Woman Detected!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    gesture = detect_gesture(frame, gestureNet)
    if gesture == "Suspicious":
        cv2.putText(frame, "ALERT: Suspicious Gesture Detected!", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    if detect_anomalies(frame, gender_counts):
        cv2.putText(frame, "ALERT: Anomaly Detected!", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Women Safety Analytics", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np
from datetime import datetime
import logging
import smtplib
from email.mime.text import MIMEText

# Configure logging
logging.basicConfig(filename='women_safety.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Load models for face detection, age, gender classification, and gesture recognition
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
gestureModel = "gesture_recognition_model.h5"  # Replace with actual model if available

faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
gestureNet = None  # Replace with actual gesture model loading if available

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Email alert configuration
EMAIL_HOST = 'smtp.example.com'  # Replace with your SMTP server
EMAIL_PORT = 587
EMAIL_HOST_USER = 'your_email@example.com'
EMAIL_HOST_PASSWORD = 'your_password'
ALERT_RECEIVER = 'receiver_email@example.com'

def send_email_alert(subject, body):
    try:
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = EMAIL_HOST_USER
        msg['To'] = ALERT_RECEIVER

        with smtplib.SMTP(EMAIL_HOST, EMAIL_PORT) as server:
            server.starttls()
            server.login(EMAIL_HOST_USER, EMAIL_HOST_PASSWORD)
            server.send_message(msg)
        logging.info(f"Email alert sent: {subject}")
    except Exception as e:
        logging.error(f"Failed to send email alert: {e}")

def faceBox(faceNet, frame):
    frameHeight, frameWidth = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bboxs = []
    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detection[0, 0, i, 3] * frameWidth)
            y1 = int(detection[0, 0, i, 4] * frameHeight)
            x2 = int(detection[0, 0, i, 5] * frameWidth)
            y2 = int(detection[0, 0, i, 6] * frameHeight)
            bboxs.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    return frame, bboxs

def classify_gender(face, genderNet, genderList):
    if face.size == 0:
        return "Unknown"
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPred = genderNet.forward()
    return genderList[genderPred[0].argmax()]

def detect_gesture(frame, gestureNet):
    if gestureNet is None:
        return "Normal"
    # Placeholder for actual gesture detection
    return "Normal"

def is_lone_woman_detected(gender_counts):
    current_hour = datetime.now().hour
    if (current_hour >= 20 or current_hour <= 6) and gender_counts['Female'] == 1 and gender_counts['Male'] == 0:
        return True
    return False

def detect_anomalies(frame, gender_counts):
    # Placeholder for advanced anomaly detection
    return False

# Real-time video capture
video = cv2.VideoCapture(0)

padding = 20

try:
    while True:
        ret, frame = video.read()
        if not ret:
            print("Failed to capture video frame.")
            break

        frame, bboxs = faceBox(faceNet, frame)
        gender_counts = {"Male": 0, "Female": 0}

        if not bboxs:
            print("No faces detected.")
        
        for bbox in bboxs:
            face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
                         max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
            
            gender = classify_gender(face, genderNet, genderList)
            if gender in gender_counts:
                gender_counts[gender] += 1

            label = f"{gender}"
            cv2.rectangle(frame, (bbox[0], bbox[1] - 30), (bbox[2], bbox[1]), (0, 255, 0), -1)
            cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        if is_lone_woman_detected(gender_counts):
            alert_message = "ALERT: Lone Woman Detected!"
            cv2.putText(frame, alert_message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            logging.info(alert_message)
            send_email_alert("Lone Woman Alert", alert_message)
        
        gesture = detect_gesture(frame, gestureNet)
        if gesture == "Suspicious":
            alert_message = "ALERT: Suspicious Gesture Detected!"
            cv2.putText(frame, alert_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            logging.info(alert_message)
            send_email_alert("Suspicious Gesture Alert", alert_message)

        if detect_anomalies(frame, gender_counts):
            alert_message = "ALERT: Anomaly Detected!"
            cv2.putText(frame, alert_message, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            logging.info(alert_message)
            send_email_alert("Anomaly Alert", alert_message)

        # Show the frame in OpenCV window (if applicable)
        try:
            cv2.imshow("Women Safety Analytics", frame)
        except cv2.error as e:
            print(f"OpenCV error: {e}")

        # Check for 'q' key to break the loop
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

finally:
    video.release()
    try:
        cv2.destroyAllWindows()
    except cv2.error as e:
        print(f"OpenCV error during cleanup: {e}")
    print("Video capture released.")


No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces det

with sms 

In [1]:
import cv2
import numpy as np
from datetime import datetime
import logging
from twilio.rest import Client

# Configure logging
logging.basicConfig(filename='women_safety.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Twilio SMS configuration
TWILIO_ACCOUNT_SID = 'your_account_sid'
TWILIO_AUTH_TOKEN = 'your_auth_token'
TWILIO_PHONE_NUMBER = 'your_twilio_phone_number'
ALERT_RECEIVER_PHONE = '+917988123720'  # Replace with your phone number

# Initialize Twilio client
twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

def send_sms_alert(body):
    try:
        message = twilio_client.messages.create(
            body=body,
            from_=TWILIO_PHONE_NUMBER,
            to=ALERT_RECEIVER_PHONE
        )
        logging.info(f"SMS alert sent: {body}")
    except Exception as e:
        logging.error(f"Failed to send SMS alert: {e}")

# Load models for face detection, age, gender classification, and gesture recognition
faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"
genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"
gestureModel = "gesture_recognition_model.h5"  # Replace with actual model if available

faceNet = cv2.dnn.readNet(faceModel, faceProto)
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
gestureNet = None  # Replace with actual gesture model loading if available

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

def faceBox(faceNet, frame):
    frameHeight, frameWidth = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], swapRB=False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bboxs = []
    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detection[0, 0, i, 3] * frameWidth)
            y1 = int(detection[0, 0, i, 4] * frameHeight)
            x2 = int(detection[0, 0, i, 5] * frameWidth)
            y2 = int(detection[0, 0, i, 6] * frameHeight)
            bboxs.append([x1, y1, x2, y2])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    return frame, bboxs

def classify_gender(face, genderNet, genderList):
    if face.size == 0:
        return "Unknown"
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    genderNet.setInput(blob)
    genderPred = genderNet.forward()
    return genderList[genderPred[0].argmax()]

def detect_gesture(frame, gestureNet):
    if gestureNet is None:
        return "Normal"
    # Placeholder for actual gesture detection
    return "Normal"

def is_lone_woman_detected(gender_counts):
    current_hour = datetime.now().hour
    if (current_hour >= 20 or current_hour <= 6) and gender_counts['Female'] == 1 and gender_counts['Male'] == 0:
        return True
    return False

def detect_anomalies(frame, gender_counts):
    # Placeholder for advanced anomaly detection
    return False

# Real-time video capture
video = cv2.VideoCapture(0)

padding = 20

try:
    while True:
        ret, frame = video.read()
        if not ret:
            print("Failed to capture video frame.")
            break

        frame, bboxs = faceBox(faceNet, frame)
        gender_counts = {"Male": 0, "Female": 0}

        if not bboxs:
            print("No faces detected.")
        
        for bbox in bboxs:
            face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
                         max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
            
            gender = classify_gender(face, genderNet, genderList)
            if gender in gender_counts:
                gender_counts[gender] += 1

            label = f"{gender}"
            cv2.rectangle(frame, (bbox[0], bbox[1] - 30), (bbox[2], bbox[1]), (0, 255, 0), -1)
            cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

        if is_lone_woman_detected(gender_counts):
            alert_message = "ALERT: Lone Woman Detected!"
            cv2.putText(frame, alert_message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            logging.info(alert_message)
            send_sms_alert(alert_message)
        
        gesture = detect_gesture(frame, gestureNet)
        if gesture == "Suspicious":
            alert_message = "ALERT: Suspicious Gesture Detected!"
            cv2.putText(frame, alert_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            logging.info(alert_message)
            send_sms_alert(alert_message)

        if detect_anomalies(frame, gender_counts):
            alert_message = "ALERT: Anomaly Detected!"
            cv2.putText(frame, alert_message, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            logging.info(alert_message)
            send_sms_alert(alert_message)

        # Show the frame in OpenCV window
        try:
            cv2.imshow("Women Safety Analytics", frame)
        except cv2.error as e:
            print(f"OpenCV error: {e}")

        # Check for 'q' key to break the loop
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

finally:
    video.release()
    try:
        cv2.destroyAllWindows()
    except cv2.error as e:
        print(f"OpenCV error during cleanup: {e}")
    print("Video capture released.")


No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces detected.
No faces det